# Predicting House Prices

**Sales prices prediction using an artificial neural network in Keras**

**Supervised Learning. Regression**

Source: Ames Housing dataset ([Kaggle website](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)).  

In [ ]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ds_boost
from tensorflow import keras

log = ds_boost.logger.init(level="DEBUG", save_log=False)

ds_boost.set_parent_execution_path()
ds_boost.info_system()
sns.set_palette("GnBu_d")
# ds_boost.reproducible(seed=0) # Setup reproducible results from run to run using Keras

%matplotlib inline
%load_ext autoreload
%autoreload 2

## 1. Data Processing and Exploratory Data Analysis

In [ ]:
data_path = "data/house_prices_data.csv"
target = ["SalePrice"]

df_original = pd.read_csv(data_path)

### Explore the data

In [ ]:
ds_boost.info_data(df_original, target)

In [ ]:
df_original.head(3)

#### Missing values

In [ ]:
high_missing = ds_boost.missing(df_original, limit=0.4, plot=True)
high_missing

### Transform the data

#### Remove irrelevant features

In [ ]:
df = df_original.copy()  # modified dataset

# remove non-significant and high-missing features
droplist = ["Id"] + high_missing

assert not set(droplist).intersection(set(target)), "Targets cannot be dropped"

df.drop(droplist, axis="columns", inplace=True)

#### Classify variables

Change categorical variables as dtype 'categorical' and sort columns: numerical + categorical + target

In [ ]:
numerical = list(df.select_dtypes(include=[np.number]))

df = ds_boost.sort_columns_by_type(df, target, numerical=numerical)

ds_boost.get_types(df)

#### Remove low frequency values from categorical features

In [ ]:
df, dict_categories = ds_boost.remove_categories(df, target, ratio=0.01)

#### Fill missing values
Numerical -> median, categorical -> mode

In [ ]:
df = ds_boost.fill_simple(df, target)

### Visualize the data

#### Target vs some significant features

In [ ]:
g = sns.PairGrid(df, y_vars=["SalePrice"], x_vars=["LotArea", "YearBuilt"], height=5, hue="OverallQual")
g.map(plt.scatter).add_legend()
g.axes[0, 0].set_xlim(0, 20000)
plt.ylim(df["SalePrice"].min(), 600000)

Lower sale prices are usually found in very low overall quality houses, with less dependency on its size and the year of construction. These three features alone are insufficient to make a good price prediction.

#### Categorical features

In [ ]:
ds_boost.show_categorical(df, sharey=True)

#### Target vs Categorical features

In [ ]:
ds_boost.show_target_vs_categorical(df, target)

#### Numerical features

In [ ]:
ds_boost.show_numerical(df, kde=True)

#### Target vs Numerical features

In [ ]:
ds_boost.show_target_vs_numerical(df, target, point_size=20, jitter=0.2, fit_reg=False)

#### Correlation between numerical features and target

In [ ]:
ds_boost.correlation(df, target)

## 2. Neural Network model

### Select the features

In [ ]:
droplist = []  # features to drop

# For the model 'data' instead of 'df'
data = df.copy()
data.drop(droplist, axis="columns", inplace=True)
data.head(3)

### Scale numerical variables
Shift and scale numerical variables to a standard normal distribution. The scaling factors are saved to be used for predictions.

In [ ]:
data, scale_param = ds_boost.scale(data)

### Create dummy features
Replace categorical features (no target) with dummy features

In [ ]:
data, dict_dummies = ds_boost.replace_by_dummies(data, target)

model_features = [f for f in data if f not in target]  # sorted neural network inputs

data.head(3)

### Split the data into training, validation, and test sets
Data leakage: Test set hidden when training the model, but seen when preprocessing the dataset

In [ ]:
test_size = 0.2
val_size = 0.1
random_state = 9

x_train, y_train, x_val, y_val, x_test, y_test = ds_boost.data_split_for_ml_with_val(
    data, target, test_size=test_size, val_size=val_size, random_state=random_state
)
x_train, y_train, x_val, y_val, x_test, y_test = ds_boost.convert_to_tensors(
    (x_train, y_train, x_val, y_val, x_test, y_test)
)

One-hot encode the output not needed for regression

### Build and Train the Neural Network

In [ ]:
model_path = os.path.join("models", "house_prices.h5")

weights = weights = keras.initializers.TruncatedNormal(stddev=0.0001, seed=9)
# opt = keras.optimizers.Adam(learning_rate=0.00005)

# import legacy adam
import tensorflow as tf

opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.00005)
model = None
model = ds_boost.build_nn_reg(
    x_train.shape[1],
    y_train.shape[1],
    hidden_layers=1,
    input_nodes=x_train.shape[1] // 2,
    dropout=0.2,
    kernel_initializer=weights,
    bias_initializer=weights,
    optimizer=opt,
    summary=True,
)

callbacks = [keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, verbose=0)]

ds_boost.train_nn(
    model,
    x_train,
    y_train,
    validation_data=[x_val, y_val],
    path=model_path,
    epochs=500,
    batch_size=16,
    callbacks=callbacks,
)

from sklearn.metrics import r2_score

ypred_train = model.predict(x_train)
ypred_val = model.predict(x_val)
print("Training   R2-score: \t{:.3f}".format(r2_score(y_train, ypred_train)))
print("Validation R2-score: \t{:.3f}".format(r2_score(y_val, ypred_val)))

### Train with Cross Validation

In [ ]:
# restore training set
x_train = np.vstack((x_train, x_val))
y_train = np.vstack((y_train, y_val))

In [ ]:
from sklearn.model_selection import KFold


def cv_train_nn(x_train, y_train, n_splits):
    """Create and Train models for cross validation. Return best model"""

    skf = KFold(n_splits=n_splits, shuffle=True)

    score = []

    best_model = None
    best_loss = float("inf")

    print(f"Training {n_splits} models for Cross Validation ...")

    for train, val in skf.split(x_train[:, 0], y_train[:, 0]):
        model = None
        model = ds_boost.build_nn_reg(
            x_train.shape[1],
            y_train.shape[1],
            hidden_layers=1,
            input_nodes=x_train.shape[1] // 2,
            dropout=0.2,
            kernel_initializer=weights,
            bias_initializer=weights,
            optimizer=opt,
            summary=False,
        )

        history = ds_boost.train_nn(
            model,
            x_train[train],
            y_train[train],
            show=False,
            validation_data=(x_train[val], y_train[val]),
            epochs=500,
            batch_size=16,
            callbacks=callbacks,
        )

        val_loss = history.history["val_loss"][-1]

        score.append(val_loss)

        if val_loss < best_loss:  # save best model (fold) for evaluation and predictions
            best_model = model
            best_loss = val_loss

    print("\nCross Validation loss: {:.3f}".format(np.mean(score)))
    return best_model


model = cv_train_nn(x_train, y_train, 10)

### Evaluate the model

In [ ]:
y_pred_test = model.predict(x_test, verbose=0)
ds_boost.regression_scores(y_test, y_pred_test, return_dataframe=True, index="DNN")

### Make predictions

In [ ]:
def predict_nn(model, x_test, target):
    """Return a dataframe with actual and predicted targets in original scale"""

    for t in target:
        pred = model.predict(x_test, verbose=0)
        restore_pred = pred * scale_param[t][1] + scale_param[t][0]
        restore_pred = restore_pred.round()

        restore_y = y_test * scale_param[t][1] + scale_param[t][0]
        restore_y = restore_y.numpy().round()

        pred_label = f"Predicted_{t}"
        error_label = f"{t} error (%)"

        pred_df = pd.DataFrame({t: np.squeeze(restore_y), pred_label: np.squeeze(restore_pred)})

        pred_df[error_label] = ((pred_df[pred_label] - pred_df[t]) * 100 / pred_df[t]).round(1)

        print(t, ". Prediction error:")
        print("Mean: \t {:.2f}%".format(pred_df[error_label].mean()))
        print("Stddev:  {:.2f}%".format(pred_df[error_label].std()))
        sns.histplot(pred_df[error_label])

    return pred_df


pred_df = predict_nn(model, x_test, target)

In [ ]:
pred_df.head(10)

The error of the predicted sale prices can be modeled by a normal distribution, almost zero centered, and with a standard deviation of < 12%. Thus, ~95% of the houses are predicted within a price error < 24% respect to the actual one. 

Note: there is data leakage when removing low-frequency categorical values and scaling numerical features


### Compare with classical ML

In [ ]:
ds_boost.ml_regression(x_train, y_train, x_test, y_test)

####  Best tree-based model

In [ ]:
from lightgbm import LGBMRegressor

model = LGBMRegressor(n_jobs=-1, n_estimators=50, max_depth=17, random_state=9).fit(x_train, np.ravel(y_train))

y_pred = model.predict(x_test)

ds_boost.regression_scores(y_test, y_pred, return_dataframe=True, index="LGBM")

#### Feature importance

In [ ]:
results = ds_boost.feature_importance(model_features, model)